## **SRTF Code**

In [4]:
sample_processes=[
    {"pid": "P1", "arrival": 0, "burst": 10, "priority": 4},
    {"pid": "P2", "arrival": 1, "burst": 6, "priority": 3},
    {"pid": "P3", "arrival": 2, "burst": 11, "priority": 5},
    {"pid": "P4", "arrival": 3, "burst": 7, "priority": 4}
]

def print_schedule(results):
    print(f"{'PID':<5} {'Start':<6} {'Finish':<6} {'Waiting':<7} {'Turnaround':<10}")
    for r in results:
        print(f"{r['pid']:<5} {r['start']:<6} {r['finish']:<6} "
              f"{r['waiting']:<7} {r['turnaround']:<10}")

def srtf(sample_processes):
    sample_processes = [{**p, "remaining": p["burst"]} for p in sample_processes]
    time = 0
    complete = 0
    n = len(sample_processes)
    results = {}
    current = None

    while complete < n:
        available = [p for p in sample_processes if p["arrival"] <= time and p["remaining"] > 0]

        if not available:
            time += 1
            continue

        p = min(available, key=lambda x: x["remaining"])

        if p["pid"] not in results:
            results[p["pid"]] = {"pid": p["pid"], "start": time}

        p["remaining"] -= 1
        time += 1

        if p["remaining"] == 0:
            results[p["pid"]]["finish"] = time
            results[p["pid"]]["waiting"] = (results[p["pid"]]["finish"] - p["arrival"] - p["burst"])
            results[p["pid"]]["turnaround"] = (results[p["pid"]]["finish"] - p["arrival"])
            complete += 1

    return list(results.values())
    
print("SRTF:")
print_schedule(srtf(sample_processes))

SRTF:
PID   Start  Finish Waiting Turnaround
P1    0      23     13      23        
P2    1      7      0       6         
P4    7      14     4       11        
P3    23     34     21      32        


## **Gantt Chart & Averages Code**

In [14]:
def make_gantt_chart(results):
    results = sorted(results, key=lambda x: x["start"])

    chart = ""
    timeline = ""

    for r in results:
        length = r["finish"] - r["start"]
        bar = "-" * length
        chart += f"|{bar}"
        timeline += f"{r['start']}{' ' * length}"

    timeline += f"{results[-1]['finish']}"

    print("\nGantt Chart:")
    print(chart + "|")

    label_line = ""
    for r in results:
        length = r["finish"] - r["start"]
        label_line += "|" + r["pid"].center(length, "-")
    print(label_line + "|")

    print(timeline)

def compute_averages(results):
    total_wait = sum(r["waiting"] for r in results)
    total_turn = sum(r["turnaround"] for r in results)
    n = len(results)
    return {"avg_waiting": total_wait / n, "avg_turnaround": total_turn / n}

results = srtf(sample_processes)
make_gantt_chart(results)

avg = compute_averages(results)
print("\nAverage Waiting Time =", avg["avg_waiting"])
print("Average Turnaround Time =", avg["avg_turnaround"])


Gantt Chart:
|-----------------------|------|-------|-----------|
|-----------P1----------|--P2--|---P4--|-----P3----|
0                       1      7       23           34

Average Waiting Time = 9.5
Average Turnaround Time = 18.0
